In [ ]:
import pandas as pd
import requests
import os
import shutil
from tqdm import tqdm

In [ ]:
classes = ['new', 'old']


def download_img(url, file_path):
    r = requests.get(url)
    with open(file_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)


def download(root):
    for c in classes:
        folder_path = os.path.join(root, c)
        try:
            os.makedirs(folder_path)
        except FileExistsError:
            print(f'Folder {folder_path} has already existed.')
        folder_path = os.path.join(root, 'img', c)
        try:
            os.makedirs(folder_path)
        except FileExistsError:
            print(f'Folder {folder_path} has already existed.')

    df = pd.read_csv(os.path.join(root, 'image-pairs.csv'))
    print(df.head())

    # TODO: dowload with multithread
    for c in classes:
        for i, url in tqdm(enumerate(df[c]), total=len(df[c])):
            filename = url.split('/')[-1]
            filepath = os.path.join(root, 'img', c, filename)
            if not os.path.exists(os.path.join(root, 'img', c, filename)):
                download_img(url, filepath)
            
            # copy downloaded file to sequential filename
            image_name, ext = filename.split('.')
            new_filename = '{:04d}.{}'.format(i, ext)
            new_filepath = os.path.join(root, c, new_filename)
            shutil.copy2(filepath, new_filepath)

In [ ]:
ROOT = './data'
download(ROOT)